## Virtual file system support

Pyolite mounts a virtual `/drive` directory which allows access to files that are provided by JupyterLab.

In [ ]:
import os

In [ ]:
os.getcwd()

### Make directory

In [ ]:
os.path.isdir("dir")

In [ ]:
os.mkdir("dir")

In [ ]:
# os.path.isdir("dir")

### List directory content

In [ ]:
os.listdir(os.getcwd())

### Remove

In [ ]:
os.rmdir('dir')

In [ ]:
os.listdir(os.getcwd())

### Reading/writing file

In [ ]:
with open('test.txt', 'r') as fobj:
    print(fobj.read())

In [ ]:
with open('test.txt', 'w') as fobj:
    fobj.write('Hello, this is me again! I write this from the kernel worker')

In [ ]:
with open('test.txt', 'r') as fobj:
    print(fobj.read())

### Replace

In [ ]:
os.path.exists('test.txt')

In [ ]:
os.rename("test.txt", "test_rename.txt")

In [ ]:
os.path.exists('test.txt')

In [ ]:
os.listdir(os.getcwd())